In [1]:
import sqlite3

print(sqlite3.version) # SQLite3 모듈의 버전
print(sqlite3.sqlite_version) # SQLite의 버전

2.6.0
3.35.4


In [2]:
import sqlite3
def create(): # 테이블 생성
    conn=sqlite3.connect('books.db') # 데이터베이스 커넥션 생성
    cursor=conn.cursor() # 커서 생성, 커서 : 레코드를 탐색하는 객체
    cursor.execute('''create table if not exists books(
        title text,
        publish_date text,
        publisher text,
        pages integer,
        recommend integer
        )''')
    conn.close() # 커넥션 닫기

create()    

In [3]:
import sqlite3
def insert(): # 레코드 입력
    conn=sqlite3.connect('books.db')
    cursor=conn.cursor()
    cursor.execute("insert into books values ('Java','2018-02-28','길벗',500,10)") # 값을 직접 입력
    
    # 튜플을 이용한 데이터 입력
    sql='insert into books values (?,?,?,?,?)' 
    cursor.execute(sql, ('Python','2018-03-04','한빛',584,20)) # 튜플 1개
    
    items=[('빅데이터','2014-07-02','삼성',296,11), ('안드로이드','2010-02-10','영진',526,20), ('Spring','2013-12-12','에이콘',248,15)]
    cursor.executemany(sql, items) # 튜플 여러 개
    conn.commit()
    conn.close()

insert()

In [7]:
import sqlite3
def list():
    conn=sqlite3.connect('books.db')
    cursor=conn.cursor()
    cursor.execute('select * from books')
    books=cursor.fetchall() # fetchall : 모든 레코드를 읽어서 리스트로 저장
    for book in books:
        print(book)
    conn.close()
list()

def update():
    conn=sqlite3.connect('books.db')
    cursor=conn.cursor()
    sql='update books set recommend=? where title=?'
    cursor.execute(sql, (100,'Java'))
    conn.commit()
    conn.close()

update()
print()
list()

('Java', '2018-02-28', '길벗', 500, 10)
('Python', '2018-03-04', '한빛', 584, 20)
('빅데이터', '2014-07-02', '삼성', 296, 11)
('안드로이드', '2010-02-10', '영진', 526, 20)
('Spring', '2013-12-12', '에이콘', 248, 15)

('Java', '2018-02-28', '길벗', 500, 100)
('Python', '2018-03-04', '한빛', 584, 20)
('빅데이터', '2014-07-02', '삼성', 296, 11)
('안드로이드', '2010-02-10', '영진', 526, 20)
('Spring', '2013-12-12', '에이콘', 248, 15)


In [8]:
def delete():
    conn=sqlite3.connect('books.db')
    cursor=conn.cursor()
    sql="delete from books where publisher='삼성'"
    cursor.execute(sql)
    conn.commit()
    conn.close()

list()
delete()
print()
list()

('Java', '2018-02-28', '길벗', 500, 100)
('Python', '2018-03-04', '한빛', 584, 20)
('빅데이터', '2014-07-02', '삼성', 296, 11)
('안드로이드', '2010-02-10', '영진', 526, 20)
('Spring', '2013-12-12', '에이콘', 248, 15)

('Java', '2018-02-28', '길벗', 500, 100)
('Python', '2018-03-04', '한빛', 584, 20)
('안드로이드', '2010-02-10', '영진', 526, 20)
('Spring', '2013-12-12', '에이콘', 248, 15)


In [9]:
import sqlite3
conn=sqlite3.connect('c:/data/db/sales.db')
sql='''
create table if not exists sales(
    customer varchar(20),
    product varchar(50),
    price float,
    date date
)'''
conn.execute(sql)
data=[('김철수','배',10000,'2018-01-02'),
('박민수','사과',20000,'2018-01-15'),
('최인규','포도',15000,'2018-02-03'),
('정인택','자두',30000,'2018-02-20'),
]
sql='insert into sales values(?,?,?,?)'
conn.executemany(sql,data)
conn.commit()
cursor=conn.execute("select * from sales")
rows=cursor.fetchall()
count=0
for row in rows:
    print(row)
    count+=1
print(f'rows : {count}')

('김철수', '배', 10000.0, '2018-01-02')
('박민수', '사과', 20000.0, '2018-01-15')
('최인규', '포도', 15000.0, '2018-02-03')
('정인택', '자두', 30000.0, '2018-02-20')
rows : 4


In [10]:
import sqlite3
conn=sqlite3.connect('c:/data/db/sales.db')
conn.execute("update sales set price=?,date=? where customer=?",(80000,'2017-12-31','김철수'))
conn.commit()
cursor=conn.execute('select * from sales')
rows=cursor.fetchall()
for row in rows:
    output=[]
    for idx in range(len(row)):
        output.append(str(row[idx]))
    print(output)

['김철수', '배', '80000.0', '2017-12-31']
['박민수', '사과', '20000.0', '2018-01-15']
['최인규', '포도', '15000.0', '2018-02-03']
['정인택', '자두', '30000.0', '2018-02-20']


In [12]:
# csv 파일로부터 읽은 데이터를 테이블에 insert
import csv
import sqlite3
input_file="c:/data/db/input.csv"
conn=sqlite3.connect("c:/data/db/suppliers.db") # db 생성
cursor=conn.cursor()
sql="""
create table if not exists suppliers(
    supplier_name varchar(20),
    invoice_number varchar(20),
    part_number varchar(20),
    cost float,
    purchase_date date
)
"""
cursor.execute(sql)
sql="delete from suppliers"
cursor.execute(sql)

file_reader=csv.reader(open(input_file,'r'), delimiter=',') # csv 파일에서 데이터를 읽어서 테이블에 insert
header=next(file_reader,None) # 첫 라인을 읽음(제목행)
for row in file_reader:
    data=[]
    for idx in range(len(header)):
        data.append(row[idx])
    cursor.execute("insert into suppliers values (?,?,?,?,?)",data)
conn.commit()
rs=cursor.execute("select * from suppliers")
rows=rs.fetchall()
for row in rows:
    output=[]
    for idx in range(len(row)):
        output.append(row[idx])
    print(output)
conn.close()

['A', '001-1001', '2341', 500000.0, '2014-01-20']
['A', '001-1001', '2341', 500000.0, '2014-01-20']
['A', '001-1001', '5467', 750000.0, '2014-01-20']
['A', '001-1001', '5467', 750000.0, '2014-01-20']
['B', '50-9501', '7009', 250000.0, '2018-01-30']
['B', '50-9501', '7009', 250000.0, '2018-01-30']
['B', '50-9505', '6650', 125000.0, '2017-02-03']
['B', '50-9505', '6650', 125000.0, '2017-02-03']
['C', '920-4803', '3321', 615000.0, '2017-02-03']
['C', '920-4804', '3321', 615000.0, '2017-02-10']
['C', '920-4805', '3321', 615000.0, '2017-02-17']
['C', '920-4806', '3321', 615000.0, '2017-02-24']
